# xgb模型

In [1]:
import pandas as pd
import numpy as np
import math
import logging
from datetime import datetime

from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, Normalizer, StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import roc_curve, auc

from mlxtend.preprocessing import DenseTransformer
from mlxtend.feature_selection import ColumnSelector

import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [2]:
logger = logging.getLogger('ai')
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s  %(filename)s : %(levelname)s  %(message)s')

## 模型

### 数据预处理

In [3]:
dataset_alpha = pd.read_csv('../features/dataset_alpha.csv')

In [6]:
dataset_beta = pd.read_csv('../features/dataset_beta.csv')

In [7]:
continous = [
    'Distance',
    'Month_of_received', 'Day_of_received',
       'Weekday_of_received', 'Base_consume', 'Discount',
       'Discount_money', 'Coupon_type', 'Coupon_category',
    'Previous_duration', 'Next_duration', 'o1',
       'o2', 'o3', 'o4', 'o5', 'o6', 'o8', 'o7', 'o9', 'o10', 'o12',
       'o14', 'o11', 'o13', 'o16', 'o15', 'o18', 'o19', 'o20', 'o21',
       'o22', 'o23', 'o17', 'o24', 'o25', 'o26', 'o27', 'o28', 'o29',
       'o30', 'o38', 'o31', 'o39', 'o32', 'o33', 'o34', 'o35', 'o36',
       'o37', 'u1', 'u2', 'u3', 'u4', 'u5', 'u6',
       'u7', 'u8', 'u9', 'u10', 'u11', 'u12', 'u13', 'u14', 'u15', 'u16',
       'u17', 'u18', 'u19', 'u20', 'u21', 'u22', 'u23', 'u24', 'u25',
       'ucc1', 'ucc2', 'ucc3', 'ucc4', 'ucc5', 'ucc6', 'ucc7', 'ucc8',
       'ucc9', 'ucc10', 'ucc11', 'ucc12', 'uc1', 'uc2', 'uc3', 'uc4',
       'uc5', 'uc6', 'uc7', 'uc8', 'uc9', 'uc10', 'uc11', 'uc12', 'ud1',
       'ud2', 'ud3', 'ud4', 'ud5', 'ud6', 'ud7', 'ud8', 'ud9', 'ud10',
       'ud11', 'ud12', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8',
       'm9', 'm10', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9',
       'c10', 'ou1', 'ou2', 'ou3', 'ou4']

label = ['Label']

In [8]:
class MergeFeature(TransformerMixin):
    def __init__(self, df, key, prefix, pipe):
        self.df = df
        self.key = key
        self.prefix = prefix
        self.pipe = pipe
    
    def get_factor(self, df, key, prefix):
        id_df = df[[key]]
        output_df = df.drop([key], axis=1)

        self.pipe.fit(output_df)
        factors = self.pipe.transform(output_df)
        factors_df = pd.DataFrame(data=factors, columns=[prefix + '_factor_alpha', prefix + '_factor_beta'])
        factors_df[key] = id_df[key]
        return factors_df
    
    def fit(self, *args, **kwargs):
        return self
    
    def transform(self, X, **transform_params):
        return pd.merge(X, self.df, on=self.key, how='left')
#         return pd.merge(X, self.get_factor(self.df, self.key, self.prefix), on=[self.key], how='left')

In [9]:
factor_pipeline = Pipeline([
    ('pca', PCA(n_components=2)),
    ('scale', MinMaxScaler()),
])

features_pipeline = Pipeline([
    ('features', FeatureUnion([
        ('continuous', Pipeline([
            ('extract', ColumnSelector(continous)),
            ('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
#             ('scale', MinMaxScaler()),
            ('normalize', Normalizer())
        ])),
#         ('fields', Pipeline([
#             ('extract', ColumnSelector(fields)),
#             ('imputer', SimpleImputer(missing_values=np.nan,  strategy='most_frequent')),
#             ('one_hot', OneHotEncoder(categories='auto')),
#             ('to_dense', DenseTransformer())
#         ])),
    ])),
])

features_pipeline.fit(dataset_beta, dataset_beta.Label.values.ravel())

train_dataset_x = features_pipeline.transform(dataset_beta)
train_dataset_y = dataset_beta.Label.values.ravel()

valid_dataset_x = features_pipeline.transform(dataset_alpha)
valid_dataset_y = dataset_alpha.Label.values.ravel()

### 模型训练

#### 调参数

In [19]:
parameters = {
    'xgb__learn_rate': [0.001, ],
    'xgb__max_depth': [6],
    'xgb__min_child_weight': [1],
    'xgb__subsample': [0.7,],
    'xgb__colsample_bytree': [0.7,],
    'xgb__colsample_bylevel': [0.7,],
    'xgb__objective': ['rank:pairwise'],
    'xgb__n_estimators': range(100, 401, 100), # 使用1-3都可以被接受
    'xgb__gamma': [0.1,],
    'xgb__reg_alpha': [1,],
    'xgb__reg_lambda': [1,],
    'xgb__max_delta_step': [0,],
    'xgb__scale_pos_weight': [1,],
    'xgb__silent': [True],
    'xgb__eval_metric': ['auc']
}

cv = GridSearchCV(model_pipeline, parameters, scoring = 'roc_auc', n_jobs= 4)
cv.fit(train_dataset_x, train_dataset_y)

/Users/leewind/.local/share/virtualenvs/leewind-p6XO93Th/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('xgb', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.7,
       colsample_bytree=0.7, eval_metric='auc', gamma=0.1, learn_rate=0.01,
       learning_rate=0.1, max_delta_step=0, max_depth=12,
       min_child_weight=1.1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=1, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.7))]),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'xgb__learn_rate': [0.001], 'xgb__max_depth': [6], 'xgb__min_child_weight': [1], 'xgb__subsample': [0.7], 'xgb__colsample_bytree': [0.7], 'xgb__colsample_bylevel': [0.7], 'xgb__objective': ['rank:pairwise'], 'xgb__n_estimators': range(100, 401, 100), 'xgb__gamma': [0.1], 'xgb__reg_alpha': [1], 'xgb__reg_lambda': [1], 'xgb__max_delta_step': [0], 'xgb__scale_pos_weight': [

In [20]:
# 查看每组评估的具体数据
cv.cv_results_['param_xgb__n_estimators'].data

# 结果训练
cv.cv_results_['mean_train_score']

/Users/leewind/.local/share/virtualenvs/leewind-p6XO93Th/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


array([0.8917712 , 0.90391788, 0.91151292, 0.91780179])

#### 最优参数训练

In [10]:
model_pipeline = Pipeline([
    ('xgb', xgb.sklearn.XGBClassifier())
])

model_pipeline.set_params(
    xgb__learn_rate=0.01,
    xgb__max_depth=12,
    xgb__min_child_weight=1.1,
    xgb__subsample=0.7,
    xgb__colsample_bytree=0.7,
    xgb__colsample_bylevel=0.7,
#     xgb__objective='rank:pairwise',
    xgb__objective='binary:logistic',
    xgb__n_estimators=30,
    xgb__gamma=0.1,
    xgb__reg_alpha=1,
    xgb__reg_lambda=1,
    xgb__max_delta_step=0,
    xgb__scale_pos_weight=1,
    xgb__silent=True,
    xgb__eval_metric='auc'
).fit(train_dataset_x, train_dataset_y)

Pipeline(memory=None,
     steps=[('xgb', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.7,
       colsample_bytree=0.7, eval_metric='auc', gamma=0.1, learn_rate=0.01,
       learning_rate=0.1, max_delta_step=0, max_depth=12,
       min_child_weight=1.1, missing=None, n_estimators=30, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=1, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.7))])

In [11]:
class Evaluator():
    def __init__(self, df, pipe):
        self.df = df
        self.pipe = pipe
        
    def transfer_result(self, result):
        return MinMaxScaler(copy=True, feature_range=(0, 1)).fit_transform(result.reshape(-1, 1))
    
    def cal(self, dataset):
        pred = self.pipe.predict_proba(dataset)[:,1]
        logging.info(pred)
        
        self.df['Probability'] = self.transfer_result(pred)
        return self.evaluate(self.df[['Probability', 'Coupon_id', 'Label']])
    
    def describe(self):
        return self.df[['User_id', 'Coupon_id', 'Probability', 'Label']].describe()
    
    def predict(self, dataset):
        pred = self.pipe.predict_proba(dataset)[:,1]
        logging.info(pred)
        
        self.df['Probability'] = self.transfer_result(pred)
        return self.df[['User_id', 'Coupon_id', 'Date_received', 'Probability']]
    
    def evaluate(self, result_df):
        group = result_df.groupby(['Coupon_id'])
        aucs = []
        logging.info('coupon size is %d' % (len(group)))

        counter = 0
        for i in group:
            tmpdf = i[1]        
            if len(tmpdf['Label'].unique()) != 2:
                continue

            fpr, tpr, thresholds = roc_curve(tmpdf['Label'], tmpdf['Probability'], pos_label=1)
            auc_score = auc(fpr,tpr)
            aucs.append(auc_score)
            counter = counter + 1

        logging.info('coupon in cal is %d' % (counter))

        return np.average(aucs)

In [12]:
evaluator = Evaluator(dataset_alpha, model_pipeline)
evaluator.cal(valid_dataset_x)

2019-02-06 01:06:14,827  <ipython-input-11-88d5387b257b> : INFO  [0.03550993 0.36370125 0.16911688 ... 0.08443704 0.02785196 0.05655894]
2019-02-06 01:06:15,033  <ipython-input-11-88d5387b257b> : INFO  coupon size is 4800
2019-02-06 01:06:17,544  <ipython-input-11-88d5387b257b> : INFO  coupon in cal is 2048


0.7002481518223771

## 预测

In [14]:
model_pred_df = pd.read_csv('../features/lcm_submit_features.csv')

predict_dataset_x = features_pipeline.transform(model_pred_df)
predictor = Evaluator(model_pred_df, model_pipeline)
final_result_df = predictor.predict(predict_dataset_x)
final_result_df.shape

/Users/leewind/.local/share/virtualenvs/leewind-p6XO93Th/lib/python3.7/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
2019-02-02 12:42:23,332  <ipython-input-12-3fa321e3310d> : INFO  [0.04794511 0.02325486 0.0034588  ... 0.00014111 0.03200784 0.04099259]


(113640, 4)

In [15]:
final_result_df.describe()

,User_id,Coupon_id,Date_received,Probability
count,1.136400e+05,113640.000000,1.136400e+05,113640.000000
mean,3.684858e+06,9053.810929,2.016072e+07,0.045039
std,2.126259e+06,4145.873088,9.019508e+00,0.083413
min,2.090000e+02,3.000000,2.016070e+07,0.000000
25%,1.844191e+06,5023.000000,2.016071e+07,0.017546
50%,3.683266e+06,9983.000000,2.016072e+07,0.027178
75%,5.525845e+06,13602.000000,2.016072e+07,0.043449
max,7.361024e+06,14045.000000,2.016073e+07,1.000000


In [27]:
final_result_df.to_csv('/Users/leewind/Desktop/submission_20190201_1.csv', index=False, header=False)